In [130]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [131]:
df = pd.read_csv( "/home/bautista/Datos/Machine-Learning-Datos/Training.csv" )
df

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.00,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.60,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.50,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.50,USD,13357192.5,Closed Lost,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16942,8781,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16943,8786,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q2,2016,NaT,0,EUR,93015.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16944,8792,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16945,28561,Americas,NE America,1,1,1,1,0,Bureaucratic_Code_4,10/20/2015,...,Q2,2016,NaT,4,USD,2346796.88,USD,0.0,Closed Lost,Prod_Category_A_None


In [132]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [133]:
df.dropna(inplace = True)
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'],errors='coerce')
df["Year Created"] = df["Opportunity_Created_Date"].dt.year
df["Month Created"] =df["Opportunity_Created_Date"].dt.month
df["Expensive"] = np.where(df["Total_Taxable_Amount"] >= 80000, 1, 0)
df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'],errors='coerce')
df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'],errors='coerce')
df = df[df["Total_Amount"] > 0]
df["Total_Amount"] = np.log(df["Total_Amount"])
entrenamiento_by_region = df.groupby("Region").agg({"Stage" : "count"})
entrenamiento_by_region.rename(columns = {"Stage" : "Amount of Cases"}, inplace = True)
entrenamiento_won_by_region = df[df["Stage"] == "Closed Won"].groupby("Region").agg({"Stage" : "count"})
entrenamiento_won_by_region.rename(columns = {"Stage" : "Cases won"}, inplace = True)

entrenamiento_won_by_region["percentage"] = (entrenamiento_won_by_region["Cases won"] / entrenamiento_by_region["Amount of Cases"]) * 100
df = df.merge(entrenamiento_won_by_region, on = "Region")
df["Años en entregar"] = df["Delivery_Year"] - df["Year Created"]

In [134]:
short_df = df[['Region', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner",'Delivery_Year','Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr' ,'Stage' ]].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]
short_df['Decision'] = np.where(short_df['Decision'] == 'Closed Won',1,0)



short_df.describe()
short_df.dropna(inplace = True)

In [135]:
short_df

,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,EMEA,15.635076,1,12,Product_Name_99,Person_Name_18,2016,Q2,1,1,0
1,EMEA,10.940740,1,12,Product_Name_100,Person_Name_20,2016,Q1,0,0,1
2,EMEA,12.624960,1,12,Product_Name_310,Person_Name_39,2016,Q2,1,1,0
3,EMEA,11.702880,1,12,Product_Name_100,Person_Name_13,2016,Q1,0,0,1
4,EMEA,12.433069,1,12,Product_Name_100,Person_Name_18,2016,Q1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
13625,Middle East,11.708533,1,10,Product_Name_394,Person_Name_5,2019,Q1,1,1,0
13629,Middle East,11.126483,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0
13631,Middle East,11.168684,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0
13632,Middle East,12.430915,0,11,Product_Name_128,Person_Name_5,2018,Q4,0,1,0


# Cambio los OpportunityOwner < 5 a Other

In [136]:
short_df.loc[(short_df.apply(lambda x: x.map(x.value_counts()))['Opportunity_Owner']) < 5,'Opportunity_Owner'] = 'Other'
short_df

,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,EMEA,15.635076,1,12,Product_Name_99,Person_Name_18,2016,Q2,1,1,0
1,EMEA,10.940740,1,12,Product_Name_100,Person_Name_20,2016,Q1,0,0,1
2,EMEA,12.624960,1,12,Product_Name_310,Person_Name_39,2016,Q2,1,1,0
3,EMEA,11.702880,1,12,Product_Name_100,Person_Name_13,2016,Q1,0,0,1
4,EMEA,12.433069,1,12,Product_Name_100,Person_Name_18,2016,Q1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
13625,Middle East,11.708533,1,10,Product_Name_394,Person_Name_5,2019,Q1,1,1,0
13629,Middle East,11.126483,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0
13631,Middle East,11.168684,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0
13632,Middle East,12.430915,0,11,Product_Name_128,Person_Name_5,2018,Q4,0,1,0


# Cambio los Product_Name < 5 a Other

In [137]:
short_df["Product_Name"] = np.where(short_df.groupby('Product_Name')["Product_Name"].transform(len) > 5, short_df["Product_Name"], "Other")
short_df["Product_Name"].value_counts()

Product_Name_100    968
Product_Name_226    835
Product_Name_231    574
Product_Name_192    437
Other               397
                   ... 
Product_Name_180      6
Product_Name_373      6
Product_Name_376      6
Product_Name_264      6
Product_Name_104      6
Name: Product_Name, Length: 237, dtype: int64

# Creo El promedio de las regiones por quarter

In [138]:
short_df['Promedio_Region_Por_Quarter'] = short_df.groupby(['Region','Delivery_Quarter'])['Total_Amount'].transform('mean')
short_df

,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter
0,EMEA,15.635076,1,12,Product_Name_99,Person_Name_18,2016,Q2,1,1,0,12.022717
1,EMEA,10.940740,1,12,Product_Name_100,Person_Name_20,2016,Q1,0,0,1,12.135386
2,EMEA,12.624960,1,12,Product_Name_310,Person_Name_39,2016,Q2,1,1,0,12.022717
3,EMEA,11.702880,1,12,Product_Name_100,Person_Name_13,2016,Q1,0,0,1,12.135386
4,EMEA,12.433069,1,12,Product_Name_100,Person_Name_18,2016,Q1,0,0,1,12.135386
...,...,...,...,...,...,...,...,...,...,...,...,...
13625,Middle East,11.708533,1,10,Other,Person_Name_5,2019,Q1,1,1,0,11.749076
13629,Middle East,11.126483,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0,12.515187
13631,Middle East,11.168684,0,10,Product_Name_464,Person_Name_5,2018,Q4,1,1,0,12.515187
13632,Middle East,12.430915,0,11,Product_Name_128,Person_Name_5,2018,Q4,0,1,0,12.515187


# Ordeno por Anio, OpportunityOwner y Quarter

In [139]:
short_df['Promedio_Owner_Por_Year_And_Quarter'] = short_df.groupby(['Delivery_Year','Opportunity_Owner','Delivery_Quarter'])['Total_Amount'].transform('mean')
short_df = short_df.sort_values(['Opportunity_Owner','Delivery_Year', 'Delivery_Quarter'])
short_df.head(25)

,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter
489,EMEA,13.271783,0,2,Other,Other,2016,Q1,0,1,0,12.135386,13.271783
404,EMEA,12.753460,0,2,Product_Name_210,Other,2017,Q2,1,1,0,12.022717,12.753460
458,EMEA,17.332958,0,3,Product_Name_432,Other,2017,Q3,1,1,0,12.179772,17.332958
13064,APAC,13.386359,0,7,Other,Other,2018,Q4,0,0,0,11.575284,13.386359
10463,APAC,11.377793,0,2,Product_Name_91,Person_Name_11,2016,Q1,1,1,1,11.794360,11.806896
13419,APAC,11.658216,1,4,Product_Name_162,Person_Name_11,2016,Q1,0,1,0,11.794360,11.806896
13429,APAC,13.164779,1,6,Product_Name_154,Person_Name_11,2016,Q1,0,1,0,11.794360,11.806896
13444,APAC,9.564512,1,10,Product_Name_77,Person_Name_11,2016,Q1,0,0,0,11.794360,11.806896
13447,APAC,13.269182,1,10,Product_Name_96,Person_Name_11,2016,Q1,0,0,0,11.794360,11.806896
10469,APAC,15.484217,0,2,Product_Name_99,Person_Name_11,2016,Q2,0,1,0,11.445778,12.832081


# Lag_1 y Delta

In [140]:
(short_df['Delivery_Quarter'][short_df.index[1]]) == (short_df['Delivery_Quarter'][short_df.index[1]])

True

In [141]:
last_person = 'NaN'
i = 0
Lag_1 = []
for person in short_df['Opportunity_Owner']:
    if person == last_person:
        if (short_df['Delivery_Quarter'][short_df.index[i]] == short_df['Delivery_Quarter'][short_df.index[i - 1]]):
            Lag_1.append(Lag_1[i - 1])
        else:
            Lag_1.append(short_df['Promedio_Owner_Por_Year_And_Quarter'][short_df.index[i - 1]])
    else:
        Lag_1.append(np.nan)
        last_person = person
    i = i + 1
short_df['Lag_1'] = Lag_1
short_df['Delta'] = short_df['Promedio_Owner_Por_Year_And_Quarter'] - short_df['Lag_1'].fillna(0)
short_df

,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
489,EMEA,13.271783,0,2,Other,Other,2016,Q1,0,1,0,12.135386,13.271783,NaN,13.271783
404,EMEA,12.753460,0,2,Product_Name_210,Other,2017,Q2,1,1,0,12.022717,12.753460,13.271783,-0.518323
458,EMEA,17.332958,0,3,Product_Name_432,Other,2017,Q3,1,1,0,12.179772,17.332958,12.753460,4.579499
13064,APAC,13.386359,0,7,Other,Other,2018,Q4,0,0,0,11.575284,13.386359,17.332958,-3.946600
10463,APAC,11.377793,0,2,Product_Name_91,Person_Name_11,2016,Q1,1,1,1,11.794360,11.806896,NaN,11.806896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7494,Americas,14.477339,1,6,Product_Name_488,Person_Name_8,2019,Q3,1,1,1,12.738383,14.973797,15.498510,-0.524714
7495,Americas,14.033311,1,6,Product_Name_483,Person_Name_8,2019,Q3,1,1,1,12.738383,14.973797,15.498510,-0.524714
7496,Americas,14.477339,1,6,Product_Name_488,Person_Name_8,2019,Q3,1,1,1,12.738383,14.973797,15.498510,-0.524714
7752,Americas,18.017027,1,10,Product_Name_445,Person_Name_8,2019,Q3,1,1,0,12.738383,14.973797,15.498510,-0.524714


# Log

In [142]:
short_df.Total_Amount = np.log(short_df.Total_Amount + 1 - min(short_df.Total_Amount))
short_df.Lag_1 = np.log(short_df.Lag_1)
short_df['Promedio_Region_Por_Quarter'] = np.log(short_df['Promedio_Region_Por_Quarter'])
short_df['Promedio_Owner_Por_Year_And_Quarter'] = np.log(short_df['Promedio_Owner_Por_Year_And_Quarter'])
short_df.Delta = np.log(short_df.Delta + 1 - min(short_df.Delta))
short_df.describe()

,Total_Amount,Años en entregar,Month Created,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
count,13583.000000,13583.000000,13583.000000,13583.000000,13583.000000,13583.00000,13583.000000,13583.000000,13583.000000,12961.000000,13583.000000
mean,2.651080,0.255025,6.574321,2017.213208,0.569388,0.77332,0.554001,2.449497,2.444527,2.452052,2.183309
std,0.177250,0.466251,3.421165,0.800279,0.495180,0.41870,0.497094,0.098289,0.142587,0.142179,0.214954
min,0.000000,-1.000000,1.000000,2016.000000,0.000000,0.00000,0.000000,2.219173,1.336753,1.655235,0.000000
25%,2.588674,0.000000,4.000000,2017.000000,0.000000,1.00000,0.000000,2.437621,2.401642,2.409813,2.088779
50%,2.656112,0.000000,6.000000,2017.000000,1.000000,1.00000,1.000000,2.491107,2.465992,2.473553,2.159336
75%,2.753412,0.000000,10.000000,2018.000000,1.000000,1.00000,1.000000,2.499777,2.544160,2.547704,2.217261
max,3.104954,5.000000,12.000000,2019.000000,1.000000,1.00000,1.000000,2.553549,2.973900,2.852610,3.240224


In [143]:
short_df.to_csv('df_time_series.csv')

# Kaggle

In [144]:
DataFrame_test = pd.read_csv( "/home/bautista/Datos/Machine-Learning-Datos/Test/Test.csv" )
DataFrame_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,...,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,...,2019 - 10,Q4,2019,NaT,0,USD,45054.9,USD,45054.9,Prod_Category_A_None
2548,15224,Americas,SE America,1,1,1,1,0,Bureaucratic_Code_4,6/8/2018,...,2019 - 10,Q4,2019,NaT,0,USD,100122.0,USD,100122.0,Prod_Category_A_None
2549,7286,Americas,NE America,1,1,0,0,0,Bureaucratic_Code_4,8/29/2018,...,2019 - 8,Q3,2019,NaT,0,USD,143220.0,USD,143220.0,Prod_Category_A_None


In [145]:
DataFrame_test.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = DataFrame_test['Total_Amount']*0.0096
DataFrame_test.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = DataFrame_test['Total_Amount']*1.17
DataFrame_test.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = DataFrame_test['Total_Amount']*0.70
DataFrame_test.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = DataFrame_test['Total_Amount']*1.29
DataFrame_test.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [146]:
DataFrame_test.dropna(inplace = True)
DataFrame_test['Opportunity_Created_Date'] = pd.to_datetime(DataFrame_test['Opportunity_Created_Date'],errors='coerce')
DataFrame_test["Year Created"] = DataFrame_test["Opportunity_Created_Date"].dt.year
DataFrame_test["Month Created"] =DataFrame_test["Opportunity_Created_Date"].dt.month
DataFrame_test["Expensive"] = np.where(DataFrame_test["Total_Taxable_Amount"] >= 80000, 1, 0)
DataFrame_test['Planned_Delivery_Start_Date'] = pd.to_datetime(DataFrame_test['Planned_Delivery_Start_Date'],errors='coerce')
DataFrame_test['Planned_Delivery_End_Date'] = pd.to_datetime(DataFrame_test['Planned_Delivery_End_Date'],errors='coerce')
DataFrame_test = DataFrame_test[DataFrame_test["Total_Amount"] > 0]


DataFrame_test["Años en entregar"] = DataFrame_test["Delivery_Year"] - DataFrame_test["Year Created"]

In [147]:
DataFrame_test = DataFrame_test[['Opportunity_ID','Region', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner",'Delivery_Year','Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr']]
DataFrame_test = DataFrame_test.drop_duplicates('Opportunity_ID',keep = 'last')
DataFrame_test

,Opportunity_ID,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
2,10689,EMEA,126555.0,0,4,Product_Name_241,Person_Name_13,2019,Q2,1,1
7,10690,EMEA,112464.0,0,4,Product_Name_322,Person_Name_13,2019,Q2,1,1
8,10691,Americas,21037.5,0,4,Product_Name_484,Person_Name_9,2019,Q4,1,1
14,10692,Americas,228327.0,0,4,Product_Name_473,Person_Name_8,2019,Q4,1,1
15,10693,Americas,5752.5,0,4,Product_Name_241,Person_Name_64,2019,Q2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2546,12364,EMEA,124740.0,0,4,Product_Name_447,Person_Name_13,2019,Q2,1,1
2547,12365,Americas,45054.9,0,4,Product_Name_477,Person_Name_38,2019,Q4,1,1
2548,12366,Americas,100122.0,0,4,Product_Name_477,Person_Name_38,2019,Q4,1,1
2549,12367,Americas,143220.0,0,4,Product_Name_281,Person_Name_9,2019,Q3,1,1


In [148]:
#for persona in DataFrame_test.drop_duplicates('Opportunity_Owner', keep = 'last')['Opportunity_Owner']:
#    flag = False
#    for chequeo in short_df.drop_duplicates('Opportunity_Owner', keep = 'last')['Opportunity_Owner']:
#        if persona == chequeo:
#            flag = True
#    if flag == False:
#        DataFrame_test.loc[DataFrame_test['Opportunity_Owner'] == persona, 'Opportunity_Owner'] = 'Other'
#DataFrame_test

In [149]:
DataFrame_test['Promedio_Region_Por_Quarter'] = DataFrame_test.groupby(['Region','Delivery_Quarter'])['Total_Amount'].transform('mean')
DataFrame_test

,Opportunity_ID,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter
2,10689,EMEA,126555.0,0,4,Product_Name_241,Person_Name_13,2019,Q2,1,1,2.562380e+05
7,10690,EMEA,112464.0,0,4,Product_Name_322,Person_Name_13,2019,Q2,1,1,2.562380e+05
8,10691,Americas,21037.5,0,4,Product_Name_484,Person_Name_9,2019,Q4,1,1,1.910773e+06
14,10692,Americas,228327.0,0,4,Product_Name_473,Person_Name_8,2019,Q4,1,1,1.910773e+06
15,10693,Americas,5752.5,0,4,Product_Name_241,Person_Name_64,2019,Q2,0,0,7.717624e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
2546,12364,EMEA,124740.0,0,4,Product_Name_447,Person_Name_13,2019,Q2,1,1,2.562380e+05
2547,12365,Americas,45054.9,0,4,Product_Name_477,Person_Name_38,2019,Q4,1,1,1.910773e+06
2548,12366,Americas,100122.0,0,4,Product_Name_477,Person_Name_38,2019,Q4,1,1,1.910773e+06
2549,12367,Americas,143220.0,0,4,Product_Name_281,Person_Name_9,2019,Q3,1,1,2.563482e+06


In [150]:
DataFrame_test['Promedio_Owner_Por_Year_And_Quarter'] = DataFrame_test.groupby(['Delivery_Year','Opportunity_Owner','Delivery_Quarter'])['Total_Amount'].transform('mean')
DataFrame_test = DataFrame_test.sort_values(['Opportunity_Owner','Delivery_Year', 'Delivery_Quarter'])
DataFrame_test.head(25)

,Opportunity_ID,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter
171,10792,EMEA,177617.70,0,1,Product_Name_201,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
226,10806,EMEA,30488.25,0,1,Product_Name_241,Person_Name_13,2019,Q1,0,1,127249.999307,59993.47619
238,10812,EMEA,96673.50,0,1,Product_Name_303,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
270,10836,EMEA,25725.00,0,1,Product_Name_182,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
282,10845,EMEA,21483.00,0,1,Product_Name_283,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
311,10864,EMEA,41418.00,0,1,Product_Name_241,Person_Name_13,2019,Q1,0,0,127249.999307,59993.47619
322,10871,EMEA,10612.80,0,1,Product_Name_322,Person_Name_13,2019,Q1,0,0,127249.999307,59993.47619
361,10900,EMEA,10478.00,0,1,Product_Name_294,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
365,10904,EMEA,20594.25,0,1,Product_Name_132,Person_Name_13,2019,Q1,1,1,127249.999307,59993.47619
366,10905,EMEA,52593.75,0,1,Product_Name_159,Person_Name_13,2019,Q1,0,1,127249.999307,59993.47619


# Lag_1 y Delta

In [151]:
last_person = 'NaN'
i = 0
Lag_1_test = []
for person in DataFrame_test['Opportunity_Owner']:
    if person == last_person:
        if (DataFrame_test['Delivery_Quarter'][DataFrame_test.index[i]]) == (DataFrame_test['Delivery_Quarter'][DataFrame_test.index[i - 1]]):
            Lag_1_test.append(Lag_1_test[i - 1])
        else:
            Lag_1_test.append(short_df['Promedio_Owner_Por_Year_And_Quarter'][DataFrame_test.index[i - 1]])
    else:
        Lag_1_test.append(np.nan)
        last_person = person
    i = i + 1
DataFrame_test['Lag_1'] = Lag_1_test
DataFrame_test['Delta'] = DataFrame_test['Promedio_Owner_Por_Year_And_Quarter'] - DataFrame_test['Lag_1'].fillna(0)
DataFrame_test

,Opportunity_ID,Region,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
171,10792,EMEA,177617.70,0,1,Product_Name_201,Person_Name_13,2019,Q1,1,1,1.272500e+05,5.999348e+04,NaN,5.999348e+04
226,10806,EMEA,30488.25,0,1,Product_Name_241,Person_Name_13,2019,Q1,0,1,1.272500e+05,5.999348e+04,NaN,5.999348e+04
238,10812,EMEA,96673.50,0,1,Product_Name_303,Person_Name_13,2019,Q1,1,1,1.272500e+05,5.999348e+04,NaN,5.999348e+04
270,10836,EMEA,25725.00,0,1,Product_Name_182,Person_Name_13,2019,Q1,1,1,1.272500e+05,5.999348e+04,NaN,5.999348e+04
282,10845,EMEA,21483.00,0,1,Product_Name_283,Person_Name_13,2019,Q1,1,1,1.272500e+05,5.999348e+04,NaN,5.999348e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,12347,Americas,4596160.80,1,4,Product_Name_487,Person_Name_9,2020,Q2,1,1,7.717624e+05,7.647092e+06,2.473553,7.647089e+06
2520,12348,Americas,10783749.90,1,4,Product_Name_475,Person_Name_9,2020,Q2,1,1,7.717624e+05,7.647092e+06,2.473553,7.647089e+06
2521,12349,Americas,7508464.43,1,4,Product_Name_432,Person_Name_9,2020,Q2,1,1,7.717624e+05,7.647092e+06,2.473553,7.647089e+06
2522,12350,Americas,10783749.90,1,4,Product_Name_475,Person_Name_9,2020,Q3,1,1,2.563482e+06,1.078375e+07,2.558681,1.078375e+07


# Log

In [152]:
DataFrame_test.Total_Amount = np.log(DataFrame_test.Total_Amount)
DataFrame_test.Lag_1 = np.log(DataFrame_test.Lag_1)
DataFrame_test.Promedio_Region_Por_Quarter = np.log(DataFrame_test.Promedio_Region_Por_Quarter)
DataFrame_test.Promedio_Owner_Por_Year_And_Quarter = np.log(DataFrame_test.Promedio_Owner_Por_Year_And_Quarter)
DataFrame_test.Delta = np.log(DataFrame_test.Delta + 1 - min(DataFrame_test.Delta))
DataFrame_test.describe()

,Opportunity_ID,Total_Amount,Años en entregar,Month Created,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
count,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1060.000000,1567.000000
mean,11554.048500,11.947143,0.031908,2.413529,2019.031908,0.493937,0.619655,13.561201,12.734706,0.904906,12.731412
std,492.205946,1.986646,0.179407,1.086323,0.179407,0.500123,0.485626,1.504236,1.591525,0.018632,1.610315
min,10689.000000,4.359014,-1.000000,1.000000,2018.000000,0.000000,0.000000,11.733856,4.359014,0.881035,0.000000
25%,11080.500000,10.930165,0.000000,1.000000,2019.000000,0.000000,0.000000,12.453862,11.671593,0.892573,11.670934
50%,11583.000000,11.708669,0.000000,2.000000,2019.000000,0.000000,1.000000,13.556432,12.416221,0.903425,12.415898
75%,11976.500000,12.827290,0.000000,3.000000,2019.000000,1.000000,1.000000,14.756877,13.432270,0.909776,13.432153
max,12368.000000,19.996115,1.000000,4.000000,2020.000000,1.000000,1.000000,17.360418,19.924758,0.984531,19.924758


In [153]:
DataFrame_test.to_csv('df_time_series_test.csv')